In [1]:
from run_bert_relation_prediction import *

## Setup directories

In [14]:
# basic parameters
os.environ['CUDA_VISIBLE_DEVICES']= '2'
data_path = "./data/wc"
data_saved_path = "./output_wc_result"
bert_model="bert-base-cased"
task_name="kg"
max_seq_length=25
eval_batch_size=32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# load precessor
processors = {"kg": KGProcessor,}
processor = processors[task_name]()

# obtain label
label_list = processor.get_relations(data_path)
num_labels = len(label_list)

# obtain entity list
entity_list = processor.get_entities(data_path)

# load model
tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=False)
model = BertForSequenceClassification.from_pretrained(data_saved_path, num_labels=num_labels)
location_detail=model.to(device)

## Train File Relation Prediction

In [16]:
def rank_accuracy(ranks):
    # check the accuracy for different hits
    max_dep = 1
    threshold = 0
    accuracy_dict = dict()
    ite = 0
    
    while threshold <= max_dep and ite <500:
        for rank in ranks:
            max_dep = max(max_dep, rank)
            if rank <= threshold:
                accuracy_dict[threshold] = accuracy_dict.get(threshold,0)+1
                
        accuracy_dict[threshold] = accuracy_dict[threshold]/len(ranks)
        threshold += 1
        ite += 1
    return accuracy_dict

In [17]:
# load train data
eval_examples = processor.get_train_examples(data_path)
eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer)

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# do predict
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Testing"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    
    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

preds = preds[0]
print(preds, preds.shape)

all_label_ids = all_label_ids.numpy()

Testing: 100%|██████████| 2500/2500 [02:45<00:00, 15.10it/s]

[[ 1.0307009  -0.8751827  -1.545766   ... -3.4059386  -4.093295
  -4.07487   ]
 [13.214246   -0.2566802  -0.93723476 ... -3.4859982  -3.6727476
  -2.056883  ]
 [ 0.57458293 -0.88107705 -1.1419075  ... -3.3871107  -3.8235722
  -3.6838179 ]
 ...
 [13.230572   -0.49823722 -0.56481314 ... -3.1396358  -3.620214
  -1.97463   ]
 [13.343311   -0.0898383  -0.91775936 ... -3.6229043  -3.5652847
  -2.1837394 ]
 [13.411948    0.04543418 -0.32135367 ... -3.4476154  -4.0423665
  -2.4703321 ]] (80000, 27)


In [18]:
# calculate the rank of the correct answer location in the 
ranks = []
filter_ranks = []
hits = []
hits_filter = []
for i in range(10):
    hits.append([])
    hits_filter.append([])

for i, pred in enumerate(preds):
    rel_values = torch.tensor(pred)
    _, argsort1 = torch.sort(rel_values, descending=True)
    argsort1 = argsort1.cpu().numpy()

    rank = np.where(argsort1 == all_label_ids[i])[0][0]
    #print(argsort1, all_label_ids[i], rank)
    ranks.append(rank)

In [19]:
accuracy_dict = rank_accuracy(ranks)
# show the accuracy for highest rank candit
accuracy_dict[0]

0.9946375

## Dev File Relation Prediction

In [20]:
# load dev data
eval_examples = processor.get_dev_examples(data_path)
eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer)

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# do predict
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Testing"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    
    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

preds = preds[0]
print(preds, preds.shape)

all_label_ids = all_label_ids.numpy()

Testing: 100%|██████████| 313/313 [00:20<00:00, 15.34it/s]

[[ 0.44482747 -1.1770259  -0.76383555 ... -2.8228853  -3.4480188
  -3.615299  ]
 [12.977062   -0.5456654  -1.4575281  ... -3.1515899  -3.587447
  -2.2301178 ]
 [ 0.9858337  -0.38104838 -0.893118   ... -0.9728924  -0.69301426
  -1.2253376 ]
 ...
 [ 0.35629842 -0.91432303 -1.0345154  ... -2.9057736  -3.9165428
  -3.7044432 ]
 [ 1.1982713  -1.240171   -0.9476489  ... -3.6007466  -3.8590267
  -3.6919255 ]
 [13.138125   -0.42995504 -0.79695964 ... -3.5715022  -3.910891
  -2.2566068 ]] (10000, 27)


In [21]:
# calculate the rank of the correct answer location in the 
ranks = []
filter_ranks = []
hits = []
hits_filter = []
for i in range(10):
    hits.append([])
    hits_filter.append([])

for i, pred in enumerate(preds):
    rel_values = torch.tensor(pred)
    _, argsort1 = torch.sort(rel_values, descending=True)
    argsort1 = argsort1.cpu().numpy()

    rank = np.where(argsort1 == all_label_ids[i])[0][0]
    #print(argsort1, all_label_ids[i], rank)
    ranks.append(rank)

In [22]:
accuracy_dict = rank_accuracy(ranks)
# show the accuracy for highest rank candit
accuracy_dict[0]

0.9942

## Test File Relation Prediction

In [23]:
# load test data
eval_examples = processor.get_test_examples(data_path)
eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer)

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# do predict
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Testing"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    
    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

preds = preds[0]
print(preds, preds.shape)

all_label_ids = all_label_ids.numpy()

Testing: 100%|██████████| 313/313 [00:20<00:00, 15.45it/s]

[[13.168325   -0.49104282 -1.2100158  ... -3.2562113  -3.8417802
  -1.9294806 ]
 [ 0.8387841  -1.2297618  -1.4677217  ... -2.0106244  -3.3389864
  -2.633492  ]
 [13.231341   -0.53429186 -0.46936047 ... -3.2200503  -4.0137944
  -1.8392601 ]
 ...
 [13.291885   -0.8537414  -0.24956194 ... -3.67075    -3.7896547
  -2.2077584 ]
 [13.464903   -0.72202873 -0.7406412  ... -3.1743302  -3.9518793
  -2.504445  ]
 [ 2.6166244  -1.5206773  -1.5587469  ... -4.4415092  -4.412207
  -4.640136  ]] (10000, 27)


In [24]:
# calculate the rank of the correct answer location in the 
ranks = []
filter_ranks = []
hits = []
hits_filter = []
for i in range(10):
    hits.append([])
    hits_filter.append([])

for i, pred in enumerate(preds):
    rel_values = torch.tensor(pred)
    _, argsort1 = torch.sort(rel_values, descending=True)
    argsort1 = argsort1.cpu().numpy()

    rank = np.where(argsort1 == all_label_ids[i])[0][0]
    #print(argsort1, all_label_ids[i], rank)
    ranks.append(rank)

In [25]:
accuracy_dict = rank_accuracy(ranks)
# show the accuracy for highest rank candit
accuracy_dict[0]

0.9949